<a href="https://colab.research.google.com/github/JonatanSiracusa/download-historical-series/blob/main/download_hist_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Historical Prices

This program retrieves financial asset prices from Yahoo Finance, calculates simple and continuous returns, and determines volatility based on adjusted prices.  
Additionally, it allows exporting the processed data for further analysis.

Before running the program, users must configure the necessary parameters, which are detailed below.


## Project Process Overview

The following key steps will be followed in the implementation of this ***Project***:

1. **Kick-off**: Importing libraries, setting up variables, and defining essential functions.

2. **Data Loading**: Retrieving the required datasets.

3. **Data Cleaning**: Removing inconsistencies and handling missing values.

4. **Data Transformation**: Structuring data and performing calculations for further analysis.

5. **Results saving**: Storing the processed data for further use.



## Program Parameter Configuration

### 1. Selecting Assets

*	*`TICKERS`:* A list of asset symbols to be analyzed. These names will also be used to reference the assets later in the analysis.
	*	**Example**:
		```py
		TICKERS = ["BBAR", "BMA", "VALO"]
		```


*	*`TICKERS_YF`:* A list of Yahoo Finance codes for the selected `TICKERS`.
	*	**IMPORTANT**: The codes must be in the same order as the `TICKERS`.
	*	**Example**:
		```py
		TICKERS_YF = ["BBAR.BA", "BMA.BA", "VALO.BA"]
		```


### 2. Date Range

*	*`START_DATE`:* The start date of the data series in `YYYY-MM-DD` format.
	*	**Example**:
		```py
		START_DATE = "2023-01-01"
		```


*	*`END_DATE`:* The end date of the data series in `YYYY-MM-DD` format.
	*	If left **empty** (`""`), it will **automatically use the current date**.
	*	**Example**:
		```py
		END_DATE = "2024-12-31"
		```
		Or to fetch data up to today
		```py
		END_DATE = ""
		```


### 3. Data Export Settings

*	*`EXPORT_DATA`:* Specifies whether to export the data to **.csv** and **.xlsx files**. 
	*	Possible values:
		*	`True`: Exports the files.
		*	`False`: Does not export the files.
	*	**Example**:
		```py
		EXPORT_DATA = True
		```


*	*`OUTPUT_NAME_1`:* Filename for saving the ***adjusted closing prices (Adj Close)*** of the tickers.
	*	**Example**:
		```py
		OUTPUT_NAME_1 = "adjusted_prices"
		```


*	*`OUTPUT_NAME_2`:* Filename for also saving the ***calculated returns and volatility***.
	*	**Example**:
		```py
		OUTPUT_NAME_2 = "returns_volatility"
		```


### Complete Configuration Example

```py
TICKERS = ["BBAR", "BMA", "VALO"]
TICKERS_YF = ["BBAR.BA", "BMA.BA", "VALO.BA"]
START_DATE = "2023-01-01"
END_DATE = ""
EXPORT_DATA = True
OUTPUT_NAME_1 = "adjusted_prices"
OUTPUT_NAME_2 = "returns_volatility"
```


***************************



# 1. Kick-off: Libraries Importing, Variables Setup and Functions

In [1]:
import numpy as np
import pandas as pd
import copy
import scipy
from scipy import stats
from scipy.stats import gaussian_kde
from fitter import Fitter
import math
import random
import time
import datetime as dt
from datetime import datetime

import yfinance as yf

#import matplotlib.pyplot as plt
#from matplotlib.ticker import FuncFormatter
#import seaborn as sns
#sns.set_theme(style='darkgrid')
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.set_config_file(offline=True, dimensions=(800,500))

from IPython.display import display, HTML, Javascript

import warnings
warnings.simplefilter("ignore")

Setting the parameters:

In [2]:
# Medimos el tiempo de ejecucion del programa
star_time = time.time()

MARKET_DAYS_YEAR = 252
OUTPUT_NAME_1 = 'historical-Adj_prices-byma'
OUTPUT_NAME_2 = 'historical-Adj_prices_plus-byma'
EXPORT_DATA = False

TICKERS = ['Index', 'BBAR', 'BMA', 'GGAL', 'SUPV', 'VALO']
TICKERS_YF = ['^MERV', 'BBAR.BA', 'BMA.BA', 'GGAL.BA', 'SUPV.BA', 'VALO.BA']

START_DATE = '2023-12-11'
END_DATE = ''

Functions:

In [3]:
def get_data(df, categoria, ticker):
	return df.loc[:, (categoria, ticker)]


text_custom = f"""
"""
print(text_custom.lstrip())


def descargar_datos_yf(tickers, start_date=None, end_date=None, delay=1):
	if start_date is None:
		start_date = dt.datetime(2015, 1, 1)
	if end_date is None:
		end_date = dt.datetime.now()

	data_dict = {}
	for ticker in tickers:
		try:
			df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False, progress=False)
			if not df.empty:
				data_dict[ticker] = df
				print(f'Descargado: {ticker}')
			else:
				print(f'Sin datos: {ticker}')
		except Exception as e:
			print(f'Error descargando {ticker}: {e}')
		time.sleep(delay)
	
	if data_dict:
		df = pd.concat(data_dict, axis=1)
	else:
		df = pd.DataFrame()

	return df

In [ ]:
def insert_returns(df):
	"""
	Calcula las medias móviles exponenciales (EMAs) y las agrega al DataFrame.
	
	Parámetros:
	- df: DataFrame original.
	"""

	df[df < 0] = 0
	df['returns'] = np.log(df['Adj Close']).diff().fillna(0)


	# df['EMA1'] = df['Adj Close'].ewm(span=EMA1, adjust=False).mean()
	# df['EMA2'] = df['Adj Close'].ewm(span=EMA2, adjust=False).mean()
	return df


# def insert_ohlcv(df, tickers, prices, tipo_precio):
def insert_ohlcv_no_usar(prices, tickers, tipo_precio=None, dfs_existentes=None):
	tipo_precio = tipo_precio or ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
	dfs = dfs_existentes if dfs_existentes is not None else {}

	for ticker in tickers:
		try:
			df = dfs.get(ticker,pd.DataFrame(index=prices.index))
			df['Open'] = prices.loc[:, (ticker, tipo_precio[4])]
			df['High'] = prices.loc[:, (ticker, tipo_precio[2])].squeeze()
			df['Low'] = prices.loc[:, (ticker, tipo_precio[3])].squeeze()
			df['Close'] = prices.loc[:, (ticker, tipo_precio[1])].squeeze()
			df['Adj Close'] = prices.loc[:, (ticker, tipo_precio[0])].squeeze()
			df['Volume'] = prices.loc[:, (ticker, tipo_precio[5])].squeeze()

			dfs[ticker] = df
			print(f"📊 Columnas de precios agregadas a {ticker}")
		except KeyError as e:
			print(f"❌ Error al procesar {ticker}: {e}")
		# df = pd.DataFrame({
		# 	# 'Open': prices.loc[:, (ticker, tipo_precio[4])].squeeze(),
		# 	# 'High': prices.loc[:, (ticker, tipo_precio[2])].squeeze(),
		# 	# 'Low': prices.loc[:, (ticker, tipo_precio[3])].squeeze(),
		# 	# 'Close': prices.loc[:, (ticker, tipo_precio[1])].squeeze(),
		# 	# 'Adj Close': prices.loc[:, (ticker, tipo_precio[0])].squeeze(),
		# 	# 'Volume': prices.loc[:, (ticker, tipo_precio[5])].squeeze()
		# 	# 'col1': 0
		# }, index=prices.index)
		
	# df.fillna(0, inplace=True)
	# df[df < 0] = 0

	# df[df < 0] = 0
	# df['returns'] = np.log(df['Adj Close']).diff().fillna(0)
	return df


def insert_ohlcv(instancia):
	"""
	Agrega las columnas de precio OHLCVA a cada DataFrame dentro de la instancia de TickerData.
	"""
	for ticker in instancia.tickers:
		df = getattr(instancia, ticker)
		try:
			df['Open'] = instancia.prices.loc[:, (ticker, tipo_precio[4])].squeeze()
			df['High'] = instancia.prices.loc[:, (ticker, tipo_precio[2])].squeeze()
			df['Low'] = instancia.prices.loc[:, (ticker, tipo_precio[3])].squeeze()
			df['Close'] = instancia.prices.loc[:, (ticker, tipo_precio[1])].squeeze()
			df['Adj Close'] = instancia.prices.loc[:, (ticker, tipo_precio[0])].squeeze()
			df['Volume'] = instancia.prices.loc[:, (ticker, tipo_precio[5])].squeeze()
			print(f'📈 Precios agregados a {ticker}')
		except KeyError as e:
			print(f'Error al agregar precios para {ticker}: {e}')



# 2. Data Loading

In [5]:
tickers = TICKERS
tickers_yf = TICKERS_YF

start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
end_date = datetime.now() if not END_DATE else datetime.strptime(END_DATE, "%Y-%m-%d")

raw_data = descargar_datos_yf(tickers_yf, start_date, end_date)
prices = raw_data.copy()

prices.index = prices.index.strftime('%Y-%m-%d')
prices.index = pd.to_datetime(prices.index)

print('Index type:', prices.index.dtype)
prices.head()

Descargado: ^MERV
Descargado: BBAR.BA
Descargado: BMA.BA
Descargado: GGAL.BA
Descargado: SUPV.BA
Descargado: VALO.BA
Index type: datetime64[ns]


^MERV                                                    \
Price       Adj Close      Close       High       Low       Open Volume   
Ticker          ^MERV      ^MERV      ^MERV     ^MERV      ^MERV  ^MERV   
Date                                                                      
2023-12-11   976823.0   976823.0   981684.0  932700.0   941830.0      0   
2023-12-12  1010022.0  1010022.0  1020221.0  971246.0   976823.0      0   
2023-12-13  1003484.0  1003484.0  1084545.0  972811.0  1010022.0      0   
2023-12-14   989696.0   989696.0  1027984.0  987233.0  1003484.0      0   
2023-12-15   925658.0   925658.0   994546.0  921389.0   989696.0      0   

                BBAR.BA                                ... SUPV.BA  \
Price         Adj Close        Close     High     Low  ...    High   
Ticker          BBAR.BA      BBAR.BA  BBAR.BA BBAR.BA  ... SUPV.BA   
Date                                                   ...           
2023-12-11  1575.508545  1875.699951  1890.25  1780.0  ...   794.5   
2023-12-12  1558.793579  1855.800049  1930.00  1790.0  ...   850.0   
2023-12-13  1586.386108  1888.650024  1948.50  1785.0  ...   815.0   
2023-12-14  1561.565308  1859.099976  1924.00  1830.0  ...   850.0   
2023-12-15  1433.345703  1706.449951  1890.00  1683.0  ...   805.0   

                                               VALO.BA                  \
Price              Low        Open  Volume   Adj Close   Close    High   
Ticker         SUPV.BA     SUPV.BA SUPV.BA     VALO.BA VALO.BA VALO.BA   
Date                                                                     
2023-12-11  733.049988  749.099976  305292  196.943497  209.25   211.5   
2023-12-12  760.000000  785.000000  689444  209.178940  222.25   226.0   
2023-12-13  750.049988  764.000000  484670  211.531921  224.75   233.0   
2023-12-14  778.049988  812.000000  343853  213.178986  226.50   230.0   
2023-12-15  727.250000  796.000000  212659  204.943588  217.75   230.0   

                                     
Price          Low    Open   Volume  
Ticker     VALO.BA VALO.BA  VALO.BA  
Date                                 
2023-12-11  193.00  193.00   547334  
2023-12-12  210.00  210.00  2231654  
2023-12-13  215.50  220.00   665933  
2023-12-14  223.25  224.00   467792  
2023-12-15  217.00  225.25  1385657  

[5 rows x 36 columns]

# 3. Data Cleaning

En esta etapa realizaremos lo siguiente:

1.	Modificamos los nombres de los Tickers, para mayor comodidad.

2.	Creamos una Class que al instanciarla reliza lo siguiente:
	*	Crea un DF por cada activo especificado en la lista `tickers`. Se accede a cada uno de ellos por medio de `class_instance.ticker`.

	*	Trata los missing values y los valores negativos.
	*	`class_instance.create_returns_df()`: Crea un nuevo DF solo con los returns de todos los activos.
	*	`class_instance.create_returns_volat_df(window=40)`: Crea un nuevo DF solo con los returns y volatilidad de todos los activos.
	*	`class_instance.list_tickers()`: Muestra los tickers y DF disponibles.
	*	`class_instance.add_columns(function, **kwargs)`: Agrega columnas al DF de cada activo llamando a una Function externa. Esto nos posibilita utilizar Functions fuera de la Class para agregar comportamientos a la Class y que se apliquen a todos los DF (por ejemplo, calculo de medias).

3.	Creamos los siguientes DF:
	*	Todos los activos con precios OHLCV y Adj Close. 
	*	Retornos.
	*	Retornos y Volatilidades.

4.	Hacemos un Checkpoint y guardamos lo realizado hasta el momento.

In [6]:
#prices = raw_data.copy()

Cambiamos los nombres de los activos.

In [7]:
for ticker_y, name in zip(tickers_yf, tickers):
	prices.rename(columns={ticker_y: name}, inplace=True)

df = prices
print("\nNiveles del índice en columnas:")
for i, name in enumerate(df.columns.names):
	print(f"Nivel {i} ({name}): {df.columns.unique(level=i).tolist()}")

prices.head()


Niveles del índice en columnas:
Nivel 0 (None): ['Index', 'BBAR', 'BMA', 'GGAL', 'SUPV', 'VALO']
Nivel 1 (Price): ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
Nivel 2 (Ticker): ['Index', 'BBAR', 'BMA', 'GGAL', 'SUPV', 'VALO']


Index                                                    \
Price       Adj Close      Close       High       Low       Open Volume   
Ticker          Index      Index      Index     Index      Index  Index   
Date                                                                      
2023-12-11   976823.0   976823.0   981684.0  932700.0   941830.0      0   
2023-12-12  1010022.0  1010022.0  1020221.0  971246.0   976823.0      0   
2023-12-13  1003484.0  1003484.0  1084545.0  972811.0  1010022.0      0   
2023-12-14   989696.0   989696.0  1027984.0  987233.0  1003484.0      0   
2023-12-15   925658.0   925658.0   994546.0  921389.0   989696.0      0   

                   BBAR                                ...   SUPV              \
Price         Adj Close        Close     High     Low  ...   High         Low   
Ticker             BBAR         BBAR     BBAR    BBAR  ...   SUPV        SUPV   
Date                                                   ...                      
2023-12-11  1575.508545  1875.699951  1890.25  1780.0  ...  794.5  733.049988   
2023-12-12  1558.793579  1855.800049  1930.00  1790.0  ...  850.0  760.000000   
2023-12-13  1586.386108  1888.650024  1948.50  1785.0  ...  815.0  750.049988   
2023-12-14  1561.565308  1859.099976  1924.00  1830.0  ...  850.0  778.049988   
2023-12-15  1433.345703  1706.449951  1890.00  1683.0  ...  805.0  727.250000   

                                      VALO                                 \
Price             Open  Volume   Adj Close   Close   High     Low    Open   
Ticker            SUPV    SUPV        VALO    VALO   VALO    VALO    VALO   
Date                                                                        
2023-12-11  749.099976  305292  196.943497  209.25  211.5  193.00  193.00   
2023-12-12  785.000000  689444  209.178940  222.25  226.0  210.00  210.00   
2023-12-13  764.000000  484670  211.531921  224.75  233.0  215.50  220.00   
2023-12-14  812.000000  343853  213.178986  226.50  230.0  223.25  224.00   
2023-12-15  796.000000  212659  204.943588  217.75  230.0  217.00  225.25   

                     
Price        Volume  
Ticker         VALO  
Date                 
2023-12-11   547334  
2023-12-12  2231654  
2023-12-13   665933  
2023-12-14   467792  
2023-12-15  1385657  

[5 rows x 36 columns]

Necesitamos que este notebook sea escalable, por lo que vamos a crear una ***Class*** para poder crear tantos DF como Tickers ingresemos:

In [8]:
tipo_precio = df.columns.unique(level=1).tolist()
tipo_precio

['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

Creamos la Class.

In [9]:
class TickerData:
	def __init__(self, tickers, prices, tipo_precio=None):
		self.tickers = tickers
		self.prices = prices
		self.tipo_precio = tipo_precio or ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
		# self.returns = None		# inicalizamos el DF de returns

		for ticker in tickers:
			df = pd.DataFrame({
				# 'Open': prices.loc[:, (ticker, tipo_precio[4])].squeeze(),
				# 'High': prices.loc[:, (ticker, tipo_precio[2])].squeeze(),
				# 'Low': prices.loc[:, (ticker, tipo_precio[3])].squeeze(),
				# 'Close': prices.loc[:, (ticker, tipo_precio[1])].squeeze(),
				# 'Adj Close': prices.loc[:, (ticker, tipo_precio[0])].squeeze(),
				# 'Volume': prices.loc[:, (ticker, tipo_precio[5])].squeeze()
			}, index=prices.index)			
			df.fillna(0, inplace=True)
			df[df < 0] = 0
			setattr(self, ticker, df)	# asignamos el DF como atributo de la Class
			print(f'DF de {ticker} se agrego a la clase.')


	def cargar_ohlcv(self):
		insert_ohlcv(self)


	def cargar_ohlcv_2(self):
	# def cargar_ohlcv(self, tipo_precio=None):
	# 	self.tipo_precio = tipo_precio or self.tipo_precio
		dfs_actuales = {ticker: getattr(self, ticker) for ticker in self.tickers}
		insert_ohlcv(self.prices, self.tickers, self.tipo_precio, dfs_existentes=dfs_actuales)
		# actualizamos atributos
		for ticker, df in dfs_actuales.items():
			setattr(self, ticker, df)

	
	def cargar_precios(self):
		agregar_columnas_precios(self)


	def list_tickers(self):
		""" Vemos los DF creados, correspondientes a cada ticker """
		return self.tickers
	
	
	def create_returns_df(self):
		""" Creamos un nuevo DF con los returns de todos los activos """
		returns_dict = {}

		for ticker in self.tickers:
			df = getattr(self, ticker)
			df[df < 0] = 0
			df['returns'] = np.log(df['Adj Close']).diff().fillna(0)
			returns_dict[ticker] = df['returns']
		
		self.returns = pd.DataFrame(returns_dict)
		print('DF de returns creado.')
		return self.returns
	
	
	def create_returns_volat_df(self, window=40):
		""" Creamos un nuevo DF con returns y volatilidad de todos los activos """
		returns_volat_dict = {}

		for ticker in self.tickers:
			df = getattr(self, ticker)
			df[df < 0] = 0
			df['returns'] = np.log(df['Adj Close']).diff().fillna(0)
			df['volat'] = (df['returns'].rolling(window=window).std().fillna(0)) * np.sqrt(MARKET_DAYS_YEAR)

			returns_volat_dict[ticker + '_returns'] = df['returns']
			returns_volat_dict[ticker + '_volat'] = df['volat']
		
		self.returns_volat = pd.DataFrame(returns_volat_dict)
		print(f'DF de returns y volatilidad anual con ventana de {window} ruedas creado.')
		return self.returns_volat
	

	def add_columns(self, function, **kwargs):
		"""
		Agregamos columnas a los DF originales de la Class llamando a una Function externa.

		Parámetros:
		- function: la function externa que recibe un DF y devuelve un DF modificado con nuevas cols o calculos.
		- kwargs: parámetros adicionales que pueda necesitar la function externa.

		Ejemplo de uso:
		data.add_columns(calculate_ema, EMA1=10, EMA2=20)
		"""
		for ticker in self.tickers:
			df = getattr(self, ticker)
			df = function(df, **kwargs)
			setattr(self, ticker, df)
		
		print(f'Se agregaron columnas y calculos usando {function.__name__}')



	def modify_df_tickers(self, ticker, column_name, function=None, remove=False):
		"""
		Agrega o elimina una columna en el DataFrame de un ticker específico.
		
		Parámetros:
		- ticker: Nombre del ticker (ej. 'AAPL', 'GOOGL').
		- column_name: Nombre de la columna a agregar o eliminar.
		- function: Función que genera los valores de la columna (solo si remove=False).
		- remove: Si es True, elimina la columna; si es False, la agrega/modifica.
		"""
		if not hasattr(self, ticker):
			raise ValueError(f"El ticker '{ticker}' no existe en los datos.")

		df = getattr(self, ticker)

		if remove:
			# Eliminar la columna si existe
			if column_name in df.columns:
				df.drop(columns=[column_name], inplace=True)
				print(f"📉 Columna '{column_name}' eliminada de {ticker}.")
			else:
				print(f"⚠️ La columna '{column_name}' no existe en {ticker}.")
		else:
			if function is None:
				raise ValueError("Debes proporcionar una función para generar los valores de la nueva columna.")
			
			df[column_name] = function(df)
			print(f"📈 Columna '{column_name}' agregada/modificada en {ticker}.")
	


In [11]:
data = TickerData(tickers, prices, tipo_precio)
data.BBAR.head()

DF de Index se agrego a la clase.
DF de BBAR se agrego a la clase.
DF de BMA se agrego a la clase.
DF de GGAL se agrego a la clase.
DF de SUPV se agrego a la clase.
DF de VALO se agrego a la clase.


""
Date
2023-12-11
2023-12-12
2023-12-13
2023-12-14
2023-12-15


In [13]:
# data.cargar_precios()
data.cargar_ohlcv()
data.GGAL

📈 Precios agregados a Index
📈 Precios agregados a BBAR
📈 Precios agregados a BMA
📈 Precios agregados a GGAL
📈 Precios agregados a SUPV
📈 Precios agregados a VALO


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-11,1685.0,1783.750000,1660.0,1756.349976,1609.195557,4215047
2023-12-12,1787.0,1833.750000,1733.5,1780.849976,1631.642822,1965001
2023-12-13,1830.0,1921.000000,1705.0,1827.300049,1674.201172,3411926
2023-12-14,1835.0,1903.000000,1780.0,1788.849976,1638.972534,3493588
2023-12-15,1790.0,1799.949951,1640.0,1652.500000,1514.046509,3948715
...,...,...,...,...,...,...
2025-03-28,7310.0,7340.000000,7130.0,7310.000000,7310.000000,2073542
2025-03-31,7020.0,7190.000000,6990.0,7160.000000,7160.000000,1778032
2025-04-01,7180.0,7390.000000,7140.0,7250.000000,7250.000000,1876008


In [ ]:
data.cargar_ohlcv_2()
data.BMA

📊 Columnas de precios agregadas a Index
📊 Columnas de precios agregadas a BBAR
📊 Columnas de precios agregadas a BMA
📊 Columnas de precios agregadas a GGAL
📊 Columnas de precios agregadas a SUPV
📊 Columnas de precios agregadas a VALO


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-11,2700.0,2899.0,2500.000000,2772.750000,2482.076904,150651
2023-12-12,2775.0,2905.0,2687.000000,2713.800049,2429.306885,226151
2023-12-13,2782.0,3300.0,2596.149902,2836.050049,2538.741211,191173
2023-12-14,2900.0,2989.5,2800.000000,2881.050049,2579.023438,381587
2023-12-15,2880.0,2919.0,2600.000000,2696.750000,2414.044189,380454
...,...,...,...,...,...,...
2025-03-28,10400.0,10400.0,9980.000000,10200.000000,10200.000000,332992
2025-03-31,9980.0,10200.0,9800.000000,9930.000000,9930.000000,365106
2025-04-01,10000.0,10225.0,9820.000000,10075.000000,10075.000000,519392


In [54]:
#data.add_columns(calculate_ema, EMA1=10, EMA2=20)
# data.add_columns(insert_returns)
data.add_columns(insert_ohlcv, tickers=tickers, prices=prices, tipo_precio=tipo_precio)
data.BBAR

Se agregaron columnas y calculos usando insert_ohlcv


""
Date
2023-12-11
2023-12-12
2023-12-13
2023-12-14
2023-12-15
...
2025-03-28
2025-03-31
2025-04-01


In [39]:
data.GGAL

""
Date
2023-12-11
2023-12-12
2023-12-13
2023-12-14
2023-12-15
...
2025-03-28
2025-03-31
2025-04-01


Creamos un Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifican con el sufijo *"v1"*:

In [ ]:
# Checkpoint
dataframes_string = tickers.copy()

for df_name in dataframes_string:
	df = globals()[df_name]
	globals()[f'{df_name}_v1'] = df.copy(deep=True)

In [ ]:
for ind, ticker in enumerate(tickers):
	print(f'Ticker {ind}: {ticker}')

Ticker 0: Index
Ticker 1: BBAR
Ticker 2: BMA
Ticker 3: GGAL
Ticker 4: SUPV
Ticker 5: VALO


In [ ]:
tickers = ['BBAR', 'BMA', 'GGAL', 'SUPV', 'VALO', 'Index']

# Diccionario para almacenar los DataFrames separados
df_por_ticker = {}

# Crear un DataFrame por cada ticker
for ticker in tickers:
	columnas_ticker = prices_v1.loc[:, prices_v1.columns.get_level_values(0) == ticker]
	df_por_ticker[ticker] = columnas_ticker

# Acceder a un DataFrame específico, por ejemplo, BBAR
df_bbar = df_por_ticker['BBAR']
print(df_bbar.head())

                   BBAR                                              
Price         Adj Close        Close     High     Low    Open  Volume
Ticker             BBAR         BBAR     BBAR    BBAR    BBAR    BBAR
Date                                                                 
2023-12-11  1575.508545  1875.699951  1890.25  1780.0  1796.0  349707
2023-12-12  1558.793579  1855.800049  1930.00  1790.0  1890.0  516615
2023-12-13  1586.386108  1888.650024  1948.50  1785.0  1865.0  205277
2023-12-14  1561.565308  1859.099976  1924.00  1830.0  1900.0  155469
2023-12-15  1433.345825  1706.449951  1890.00  1683.0  1853.0  165824


In [ ]:
BBAR, BMA, GGAL, SUPV, VALO, Index = [pd.DataFrame({
	'Open': get_data(prices_v1, categorias[4], tickers[i]),
	'High': get_data(prices_v1, categorias[2], tickers[i]),
	'Low': get_data(prices_v1, categorias[3], tickers[i]),
	'Close': get_data(prices_v1, categorias[1], tickers[i]),
	'Adj Close': get_data(prices_v1, categorias[0], tickers[i]),
	'returns': np.nan,
	}, index=prices_v1.index
) for i in range(len(tickers))]


TypeError: get_data() takes 3 positional arguments but 4 were given

In [ ]:
dataframes = [BBAR, BMA, GGAL, SUPV, VALO, Index]

for df in dataframes:
	df[df < 0] = 0
	df.fillna(0, inplace=True)

Seguimos trabajando sobre los DF originales, manteniendo una copia de los mismos, en caso de necesitar revertir cambios realizados.

# 4. Data Transformation

### Dataframe de cada activo

Calculamos y agregamos a cada DF de los activos, lo siguiente:

*	Retornos logaritmicos
*	Volatilidad de las ultimas 40 ruedas, anualizada.
*	Medias.

In [ ]:
# returns, volat & EMAs
window = 40
EMA1 = 4
EMA2 = 9

for df in dataframes:
	df['returns'] = np.log(df['Adj Close']).diff()
	df.fillna(0, inplace=True)

	df['volat_MA40'] = df['returns'].rolling(window=window).std() * np.sqrt(RUEDAS_ANIO)
	#df.fillna(0, inplace=True)

	df['EMA1'] = df['Adj Close'].ewm(span=EMA1, adjust=False).mean()
	df['EMA2'] = df['Adj Close'].ewm(span=EMA2, adjust=False).mean()
	df.fillna(0, inplace=True)


Creamos un Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifican con el sufijo *"v1"*:

In [ ]:
# Checkpoint
dataframes_string = tickers.copy()

for df_name in dataframes_string:
	df = globals()[df_name]
	globals()[f'{df_name}_v1'] = df.copy(deep=True)

In [16]:
GGAL

NameError: name 'GGAL' is not defined

### Dataframe de Retornos

Creamos un DF con los Retornos de todos los activos.

In [ ]:
returns = np.log(prices_v1['Adj Close']).diff().dropna()
returns

Creamos otro Checkpoint con lo realizado hasta ahora, y seguimos trabajando sobre los DF originales.

Se identifica con el sufijo *"v1"*:

In [ ]:
# Checkpoint
df = returns
df_name = 'returns'

df = globals()[df_name]
globals()[f'{df_name}_v1'] = df.copy(deep=True)

In [ ]:
correlations = returns.corr().round(2)
correlations

In [ ]:
datos = returns

mu = np.mean(datos, axis=0)
sigma = np.std(datos, ddof=1, axis=0)

In [ ]:
mu

In [ ]:
mean_values_numpy = np.mean(returns, axis=0)


print("\nMedia por columnas usando numpy:")
print(mean_values_numpy)

In [ ]:
returns_prom = returns.mean() * RUEDAS_ANIO
returns_prom

GUARDO EL CODIGO DE LA CLASE

In [ ]:
class TickerData:
	def __init__(self, tickers, prices, tipo_precio):
		self.tickers = tickers 
		# self.returns = None		# inicalizamos el DF de returns

		for ticker in tickers:
			df = pd.DataFrame({
				'Open': prices.loc[:, (ticker, tipo_precio[4])].squeeze(),
				'High': prices.loc[:, (ticker, tipo_precio[2])].squeeze(),
				'Low': prices.loc[:, (ticker, tipo_precio[3])].squeeze(),
				'Close': prices.loc[:, (ticker, tipo_precio[1])].squeeze(),
				'Adj Close': prices.loc[:, (ticker, tipo_precio[0])].squeeze(),
				'Volume': prices.loc[:, (ticker, tipo_precio[5])].squeeze()
			}, index=prices.index)
			
			df.fillna(0, inplace=True)
			df[df < 0] = 0
			
			# asignamos el DF como atributo de la Class
			setattr(self, ticker, df)
	
	def list_tickers(self):
		""" Vemos los DF creados, correspondientes a cada ticker """
		return self.tickers
	
	
	def create_returns_df(self):
		""" Creamos un nuevo DF con los returns de todos los activos """
		returns_dict = {}

		for ticker in self.tickers:
			df = getattr(self, ticker)
			df[df < 0] = 0
			df['returns'] = np.log(df['Adj Close']).diff().fillna(0)
			returns_dict[ticker] = df['returns']
		
		self.returns = pd.DataFrame(returns_dict)
		print('DF de returns creado.')
		return self.returns
	
	
	def create_returns_volat_df(self, window=40):
		""" Creamos un nuevo DF con returns y volatilidad de todos los activos """
		returns_volat_dict = {}

		for ticker in self.tickers:
			df = getattr(self, ticker)
			df[df < 0] = 0
			df['returns'] = np.log(df['Adj Close']).diff().fillna(0)
			df['volat'] = (df['returns'].rolling(window=window).std().fillna(0)) * np.sqrt(MARKET_DAYS_YEAR)

			returns_volat_dict[ticker + '_returns'] = df['returns']
			returns_volat_dict[ticker + '_volat'] = df['volat']
		
		self.returns_volat = pd.DataFrame(returns_volat_dict)
		print(f'DF de returns y volatilidad anual con ventana de {window} ruedas creado.')
		return self.returns_volat
	

	def add_columns(self, function, **kwargs):
		"""
		Agregamos columnas a los DF originales de la Class llamando a una Function externa.

		Parámetros:
		- function: la function externa que recibe un DF y devuelve un DF modificado con nuevas cols o calculos.
		- kwargs: parámetros adicionales que pueda necesitar la function externa.
		"""
		for ticker in self.tickers:
			df = getattr(self, ticker)
			df = function(df, **kwargs)
			setattr(self, ticker, df)
		
		print(f'Se agregaron columnas y calculos usando {function.__name__}')

# 5. Visualization

In [ ]:
prices_v1['Close'].normalize()

In [ ]:
tickers

## 5.1 Plot normalizado

In [ ]:
datos = prices_v1['Close'].normalize()
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
#periodo = f"{fecha_inicio.strftime('%Y-%b')} - {fecha_fin.strftime('%Y-%b')}"
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"
"""
fecha_inicio = datetime.strptime(inicio, '%Y-%m-%d').strftime('%d/%m/%Y')
fecha_fin = datetime.now().strftime('%d/%m/%Y')
"""

#eje_x = np.arange(len(datos))
eje_x = datos.index

fig_activos_norm = go.Figure()

line_props = [
	{"y": datos[tickers[0]], 'color': 'black', "dash": "solid", "text_leg": f'BBAR'},
	{"y": datos[tickers[1]], 'color': 'purple', "dash": "solid", "text_leg": f'BMA'},
	{"y": datos[tickers[2]], 'color': 'green', "dash": "solid", "text_leg": f'GGAL'},
	{"y": datos[tickers[3]], 'color': 'blue', "dash": "solid", "text_leg": f'SUPV'},
	{"y": datos[tickers[4]], 'color': 'green', "dash": "solid", "text_leg": f'VALO'},
	{"y": datos[tickers[5]], 'color': 'orange', "dash": "dot", "text_leg": f'S&P Merval'},
]

for prop in line_props:
	fig_activos_norm.add_trace(go.Scatter(
		x=eje_x,
		y=prop['y'],
		mode='lines',
		#line=dict(color=prop['color'], dash=prop['dash'], width=2),
		line=dict(dash=prop['dash'], width=2),
		opacity=0.8,
		name=prop['text_leg'],  # Agrega texto a la leyenda
		showlegend=True,
		#yaxis="y2"
	))


fig_activos_norm.update_layout(
	width=900, height=600, margin=dict(l=40, r=40, t=70, b=40),
	#title=f'S&P500: Retornos vs Value at Risk | {periodo}',
	title=dict(text=f'Rendimiento principales activos bancarios | Período {periodo}', x=0.5, y=0.95, font=dict(size=22, family="calibri")),
	font=dict(size=14, family="calibri"),
	xaxis=dict(title=f'Fechas respecto de cotizaciones diarias', type='date', tickformat='%Y-%b'),

	#yaxis=dict(title='Variacion por 100 respecto al inicio de la serie',  tickformat=".1%"), side='right', range=rango_y), 

	yaxis=dict(title=dict(text='Variacion base 100 respecto al inicio de la serie (normalizado)', font=dict(size=15), standoff=20), type='linear', side='right', tickfont=dict(size=13)),

	#yaxis2=dict(title='Value at Risk', tickformat=".1%", overlaying='y', side='right', showgrid=False, range=rango_y),
	legend=dict(bordercolor="black", borderwidth=1, x=0.01, y=0.99, xanchor="left", yanchor="top"),
	#template='plotly_white'
)

fig_activos_norm.show()

## 5.2 Plot returns vs volatilidad

In [ ]:
returns

In [ ]:
GGAL.head()

In [ ]:
datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

assets_graph = len(datos.columns)
rows_graph = (assets_graph // 2) + (assets_graph % 2)
cols_graph = 2


fig_volat = go.Figure()

fig_volat = make_subplots(rows=rows_graph, cols=cols_graph, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.08, horizontal_spacing=0.05, subplot_titles=datos.columns)


for i, col in enumerate(datos.columns):
	rows_subplot = (i // cols_graph) + 1
	cols_subplot = (i % cols_graph) + 1
	#hist_values, _ = np.histogram(datos[col].to_numpy(), bins=bin_edges, density=True)
	data_col = datos[col].to_numpy()

	fig_volat.add_trace(go.Scatter(
		x=datos.index, y=data_col, mode='lines', line=dict(width=1), name=f'{col}', showlegend=False), 
		row=rows_subplot, col=cols_subplot)


	fig_volat.add_trace(go.Scatter(
		x=datos.index, y=dataframes[i]['volat_MA40'], mode='lines', line=dict(color='orange', width=1), name=f'Volat', showlegend=True), 
		row=rows_subplot, col=cols_subplot)
	

	"""fig_hist.add_trace(go.Bar(
		x=bin_edges[:-1], y=hist_values, width=bin_width, marker=dict(opacity=0.7), name=col, showlegend=True), 
		row=rows_subplot, col=cols_subplot)

	# KDE (Curva de Densidad)
	kde = gaussian_kde(data_col)
	x_kde = np.linspace(bin_edges.min(), bin_edges.max(), 200)  # Valores X
	y_kde = kde(x_kde)  # Evaluar KDE en esos valores
	fig_hist.add_trace(go.Scatter(
		x=x_kde, y=y_kde, mode='lines', line=dict(color='red', width=2), name=f"KDE {col}", showlegend=False), 
		row=rows_subplot, col=cols_subplot)"""

fig_volat.update_layout(
	width=900, height=305*rows_graph, margin=dict(l=40, r=40, t=90, b=60),
	title=dict(text=f'Distribución de Retornos por Activo | Período {periodo}', x=0.5, y=0.97, font=dict(size=22, family="calibri")),
	font=dict(size=14, family="calibri"),
	legend=dict(bordercolor="black", borderwidth=1, x=0.99, y=0.99, xanchor="right", yanchor="top"),
	#template='plotly_white',
	#shapes=vertical_lines
	#xaxis=dict(title=f'Fechas respecto de cotizaciones diarias', type='date', tickformat='%Y-%b'),
)

#xaxis=dict(title=f'Fechas respecto de cotizaciones diarias', type='date', tickformat='%Y-%b'),
fig_volat.update_xaxes(dict(title=f'Fechas respecto de cotizaciones diarias', type='date', tickformat='%Y-%b'))

#fig_volat.update_xaxes(dict(title=dict(text='Retornos %', font=dict(size=14), standoff=10), tickfont=dict(size=12)), tickformat=".0%", dtick=0.05, )
#fig_volat.update_yaxes(title=dict(text='Frecuencia Observada', font=dict(size=14), standoff=10))

fig_volat.show()

In [ ]:
datos = prices_v1['Close'].normalize()
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

#eje_x = np.arange(len(datos))
eje_x = datos.index

fig_activos_norm = go.Figure()

line_props = [
	{"y": datos[tickers[0]], 'color': 'black', "dash": "solid", "text_leg": f'BBAR'},
	{"y": datos[tickers[1]], 'color': 'purple', "dash": "solid", "text_leg": f'BMA'},
	{"y": datos[tickers[2]], 'color': 'green', "dash": "solid", "text_leg": f'GGAL'},
	{"y": datos[tickers[3]], 'color': 'blue', "dash": "solid", "text_leg": f'SUPV'},
	{"y": datos[tickers[4]], 'color': 'green', "dash": "solid", "text_leg": f'VALO'},
	{"y": datos[tickers[5]], 'color': 'orange', "dash": "dot", "text_leg": f'S&P Merval'},
]

for prop in line_props:
	fig_activos_norm.add_trace(go.Scatter(
		x=eje_x,
		y=prop['y'],
		mode='lines',
		#line=dict(color=prop['color'], dash=prop['dash'], width=2),
		line=dict(dash=prop['dash'], width=2),
		opacity=0.8,
		name=prop['text_leg'],  # Agrega texto a la leyenda
		showlegend=True,
		#yaxis="y2"
	))

In [ ]:
dataframes[0]
dataframes_string
dataframes[2]['volat_MA40']

## 5.3 Histograma

In [ ]:
datos = returns
datos.head()

In [ ]:
"""datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"


# Histograma
hist_values, bin_edges = np.histogram(datos, bins=100, density=True)
bin_width = bin_edges[1] - bin_edges[0]  # Ancho de bin"""

In [ ]:
"""# Definir número de filas y columnas para los subgráficos
n_assets = len(datos.columns)
rows_sgraph = (n_assets // 2) + (n_assets % 2)  # 2 columnas por fila
cols_sgraph = 2"""

In [ ]:
"""# Crear la cuadrícula de subgráficos: 2 filas x 2 columnas
fig_hist = make_subplots(
	rows=rows_sgraph, cols=cols_sgraph, 
	#shared_xaxes=False,  # Ejes x compartidos
	shared_xaxes=True,  
	shared_yaxes=True,  # Ejes y compartidos
	vertical_spacing=0.1,  # Espacio vertical entre subgráficos
	horizontal_spacing=0.1,  # Espacio horizontal entre subgráficos
	subplot_titles=datos.columns  # Títulos de cada subgráfico
)"""

In [ ]:
datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

all_data = datos.to_numpy().flatten()
_, bin_edges = np.histogram(all_data, bins=50, density=True)
bin_width = bin_edges[1] - bin_edges[0]  # Ancho de bin


assets_graph = len(datos.columns)
rows_graph = (assets_graph // 2) + (assets_graph % 2)  # 2 columnas por fila
cols_graph = 2


fig_hist = go.Figure()

fig_hist = make_subplots(rows=rows_graph, cols=cols_graph, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.08, horizontal_spacing=0.05, subplot_titles=datos.columns)


for i, col in enumerate(datos.columns):
	rows_subplot = (i // cols_graph) + 1
	cols_subplot = (i % cols_graph) + 1
	hist_values, _ = np.histogram(datos[col].to_numpy(), bins=bin_edges, density=True)
	data_col = datos[col].to_numpy()

	fig_hist.add_trace(go.Bar(
		x=bin_edges[:-1], y=hist_values, width=bin_width, marker=dict(opacity=0.7), name=col, showlegend=True), 
		row=rows_subplot, col=cols_subplot)

	# KDE (Curva de Densidad)
	kde = gaussian_kde(data_col)
	x_kde = np.linspace(bin_edges.min(), bin_edges.max(), 200)  # Valores X
	y_kde = kde(x_kde)  # Evaluar KDE en esos valores
	fig_hist.add_trace(go.Scatter(
		x=x_kde, y=y_kde, mode='lines', line=dict(color='red', width=2), name=f"KDE {col}", showlegend=False), 
		row=rows_subplot, col=cols_subplot)

fig_hist.update_layout(
	width=900, height=305*rows_graph, margin=dict(l=40, r=40, t=90, b=60),
	title=dict(text=f'Distribución de Retornos por Activo | Período {periodo}', x=0.5, y=0.97, font=dict(size=22, family="calibri")),
	font=dict(size=14, family="calibri"),
	legend=dict(bordercolor="black", borderwidth=1, x=0.99, y=0.99, xanchor="right", yanchor="top"),
	template='plotly_white',
	#shapes=vertical_lines
)

fig_hist.update_xaxes(dict(title=dict(text='Retornos %', font=dict(size=14), standoff=10), tickfont=dict(size=12)), tickformat=".0%", dtick=0.05, )
fig_hist.update_yaxes(title=dict(text='Frecuencia Observada', font=dict(size=14), standoff=10))

fig_hist.show()

## 5.4 Scatter Matrix

In [ ]:
datos = returns
fecha_inicio, fecha_fin = datos.index[[0, -1]]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

assets = datos.columns
num_assets = len(assets)

font_size_axis_labels = 14  # Tamaño de letra de los nombres de los ejes
font_size_ticks = 11        # Tamaño de letra de los valores en los ejes
axis_title_padding = 10     # Espaciado entre el gráfico y las etiquetas de los ejes

espaciado_horizontal, espaciado_vertical = 0.05, 0.03
bins_hist = 20

x_min, x_max = datos.min().min(), datos.max().max()
y_max_hist = max(np.histogram(datos[asset], bins=bins_hist, density=True)[0].max() for asset in datos.columns)


fig_pairplot = make_subplots(
	rows=num_assets, cols=num_assets, 
	shared_xaxes=True, shared_yaxes=False, 
	horizontal_spacing=espaciado_horizontal, vertical_spacing=espaciado_vertical
)

def actualizar_ejes(row, col, x_range, y_range, is_histogram=False):
	show_x = row == num_assets  # Mostrar solo en la última fila
	show_y = col == 1 or is_histogram  # Mostrar en la primera columna o si es un histograma

	fig_pairplot.update_xaxes(
		range=x_range, tickformat=".0%", tickfont=dict(size=font_size_ticks),
		title_font=dict(size=font_size_axis_labels), title_standoff=axis_title_padding, 
		showticklabels=show_x, row=row, col=col
	)

	fig_pairplot.update_yaxes(
		range=y_range, tickformat=".2f" if is_histogram else ".0%",
		tickfont=dict(size=font_size_ticks), title_font=dict(size=font_size_axis_labels), 
		title_standoff=axis_title_padding, showticklabels=show_y, row=row, col=col
	)

for i, asset_x in enumerate(assets):
	for j, asset_y in enumerate(assets):
		row, col = i + 1, j + 1

		if i == j:  # Histograma
			hist_values, bin_edges = np.histogram(datos[asset_x], bins=bins_hist, density=True)
			bin_centers = bin_edges[:-1] + np.diff(bin_edges) / 2

			fig_pairplot.add_trace(go.Bar(x=bin_centers, y=hist_values, marker=dict(opacity=1), showlegend=False), row=row, col=col)
			actualizar_ejes(row, col, [x_min, x_max], [0, y_max_hist * 1.1], is_histogram=True)

		else:  # Gráfico de dispersión
			fig_pairplot.add_trace(go.Scatter(x=datos[asset_x], y=datos[asset_y], mode='markers',
									 marker=dict(size=3, opacity=0.4, color='blue'), showlegend=False), row=row, col=col)
			actualizar_ejes(row, col, [x_min, x_max], [x_min, x_max])

fig_pairplot.update_layout(
	width=1000, height=1200, margin=dict(l=40, r=40, t=110, b=40),
	title=dict(
		text=f'Pairplot: Histograma de Retornos y gráficos de Dispersión | Período {periodo}', 
		x=0.5, y=0.97, font=dict(size=22, family="calibri")
	)
)

# Nombres de los activos en la parte superior y en la primera columna
for i, asset in enumerate(assets):
	fig_pairplot.update_xaxes(title_text=asset, row=1, col=i+1, title_standoff=10, side='top')
	fig_pairplot.update_yaxes(title_text=asset, row=i+1, col=1)

fig_pairplot.show()

## 5.5 Matriz de Correlaciones

In [ ]:
correlations = returns.corr().round(2)
correlations

In [ ]:
datos = returns
fecha_inicio = datos.index[0]
fecha_fin = datos.index[-1]
periodo = f"{fecha_inicio.strftime('%d/%m/%Y')} - {fecha_fin.strftime('%d/%m/%Y')}"

correlations = datos.corr().round(2)
tickers = correlations.index.tolist()
correlation_values = correlations.values

font_size_title = 22
font_size_labels = 14  # Tamaño de la fuente para los rótulos de los ejes
font_size_values = 16  # Tamaño de la fuente de los valores en la matriz
font_size_xaxis = 14   # Tamaño de la fuente del eje X
font_size_yaxis = 14   # Tamaño de la fuente del eje Y
label_padding = 10     # Distancia entre los rótulos y la matriz
colorscale = px.colors.sequential.Viridis  # Escala de colores

fig_corr = go.Figure()
fig_corr.add_trace(
	go.Heatmap(
		z=correlation_values, x=tickers, y=tickers, colorscale=colorscale, zmin=-1, zmax=1,
		colorbar=dict(title="Correlación"),
		text=[[f"{val:.2f}" for val in row] for row in correlation_values],
		texttemplate="<span style='font-size:{}px'>%{{text}}</span>".format(font_size_values),
		hovertemplate="<b>%{x}</b> vs <b>%{y}</b>: %{z:.2f}<extra></extra>",
	)
)

fig_corr.update_layout(
	width=700, height=600, margin=dict(l=80, r=40, t=100, b=40),
	title=dict(text=f'Matriz de Correlaciones | Período {periodo}', x=0.5, y=0.97, font=dict(size=font_size_title, family="calibri")),
	font=dict(family="calibri"),
	xaxis=dict(title="Activo", side="top", title_font=dict(size=font_size_labels), title_standoff=label_padding, tickfont=dict(size=font_size_xaxis),),
	yaxis=dict(title="Activo", title_font=dict(size=font_size_labels), title_standoff=label_padding, autorange="reversed", tickfont=dict(size=font_size_yaxis), 
	)
)

fig_corr.show()

### 3.2.1. Graph 1

In [ ]:
"""inicio = '2023-12-11'
activos = ['Index', 'GGAL', 'BBAR', 'BMA', 'SUPV', 'VALO']
meses = [i for i in range(13)]

fecha_inicio = datetime.strptime(inicio, '%Y-%m-%d').strftime('%d/%m/%Y')
fecha_fin = datetime.now().strftime('%d/%m/%Y')

series_graf_1 = generar_series_graf(inicio, activos)
series_graf_1 = normalize_prices(activos, series_graf_1)
eje_x_etiquetas = crear_etiqueta_mes(meses, series_graf_1)
eje_x_titulo = f'Fechas respecto de cotizaciones diarias (período {fecha_inicio}-{fecha_fin})'
"""

In [ ]:
"""fig = go.Figure()

for col in series_graf_1.columns[1:]:
	fig.add_trace(go.Scatter(
		x=series_graf_1.index,
		y=series_graf_1[col],
		mode='lines',
		#text=['punto1', 'punto2'],
		#textposition='bottom right',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		#marker=dict(color='red', size=8, symbol='circle', line=dict(color='black', width=2)),
		opacity=0.8,
		#fill='tozeroy',
		#fill='tonexty',
		#fillcolor='rgba(0, 100, 250, 0.2)',
		name=col
	))

fig.add_trace(go.Scatter(
		x=series_graf_1.index,
		y=series_graf_1['Index'],
		mode='lines',
		line=dict(color='blue', width=3, dash='dot'),
		opacity=0.8,
		name='Index'
	))

fig.update_layout(
	width=1000, 
	height=600,
	title=dict(text='Rendimiento principales activos bancarios', x=0.5, y=0.95, font=dict(size=24)),
	xaxis=dict(
		title=dict(
			text=eje_x_titulo,
			font=dict(size=15),
			standoff=20
		),
		tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	),
	yaxis=dict(
		title=dict(
			text='Variacion por 100 respecto al inicio de la serie', 
			font=dict(size=15),
			standoff=20
		),
		type='linear', side='right', tickfont=dict(size=13)
	),
	legend=dict(
		title='Activos', x=0.02, y=0.98, font=dict(size=12), bgcolor='rgba(255, 255, 255, 0.5)'
	),
	margin=dict(l=40, r=40, t=70, b=40)
)

fig.show()"""

### 3.2.2. Graph 2

In [ ]:
"""inicio = '2023-12-11'
activos = ['GGAL', 'BBAR', 'BMA', 'VALO']
calculos=['p', 'sr', 'v40']
x_values=True
meses = [i for i in range(13)]

fecha_inicio = datetime.strptime(inicio, '%Y-%m-%d').strftime('%d/%m/%Y')
fecha_fin = datetime.now().strftime('%d/%m/%Y')

series_graf_2 = generar_series_graf(inicio, activos, calculos, x_values)
series_graf_2 = normalize_prices(activos, series_graf_2)
eje_x_etiquetas = crear_etiqueta_mes(meses, series_graf_2)
eje_x_titulo = f'Fechas respecto de cotizaciones diarias (período {fecha_inicio}-{fecha_fin})'
"""

In [ ]:
"""# Crear la cuadrícula de subgráficos: 2 filas x 2 columnas
fig = make_subplots(
	rows=2, cols=2, 
	shared_xaxes=False,  # Ejes x compartidos
	shared_yaxes=True,  # Ejes y compartidos
	vertical_spacing=0.1,  # Espacio vertical entre subgráficos
	horizontal_spacing=0.1,  # Espacio horizontal entre subgráficos
	subplot_titles=("GGAL", "BBAR", "BMA", "VALO")  # Títulos de cada subgráfico
)

# Añadir el primer gráfico (Scatter) en la posición (1,1)
for col in series_graf_2.columns[1:4]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=1, col=1
	)

# Añadir el segundo gráfico en la posición (1,2)
for col in series_graf_2.columns[4:7]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=1, col=2
	)

# Añadir el tercer gráfico en la posición (2,1)
for col in series_graf_2.columns[7:10]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=2, col=1
	)

# Añadir el cuarto gráfico en la posición (2,2)
for col in series_graf_2.columns[10:13]:
	fig.add_trace(go.Scatter(
		x=series_graf_2.index,
		y=series_graf_2[col],
		mode='lines',
		#line=dict(color='blue', width=3, dash='dash'),
		line=dict(width=2),
		opacity=0.8,
		name=col),
		row=2, col=2
	)

fig.update_layout(
	width=1100, 
	height=1400,
	title=dict(text='Rendimiento principales activos bancarios', x=0.5, y=0.97, font=dict(size=24), pad=dict(t=10)),
	#annotations=[
	#    dict(
	#        text=eje_x_titulo,
	#        x=0.5,                  # Centrado horizontalmente
	#        y=-0.09,                 # Debajo de los subgráficos
	#        xref="paper",           # Usa las coordenadas del layout global
	#        yref="paper",
	#        font=dict(size=16)      # Tamaño de la fuente del título
	#    )
	#],

	#xaxis=dict(
	#	title=dict(
	#		text=eje_x_titulo,
	#		font=dict(size=12),
	#		standoff=15
	#	),
	#	tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	#),
	#xaxis=dict(tickangle=45, tickfont=dict(size=12), tickvals=eje_x_etiquetas, ticktext=eje_x_etiquetas
	#),

	yaxis=dict(
		title=dict(
			text='Variacion por 100 respecto al inicio de la serie', 
			font=dict(size=12),
			standoff=20
		),
		type='linear', side='right', tickfont=dict(size=13)
	),
	legend=dict(
		title='Activos',
		bgcolor='rgba(255, 255, 255, 0.5)',
		orientation="h",      # Orientación horizontal
		x=0.5,                # Centrada horizontalmente
		xanchor="center",
		y=-0.10,               # Ubicada debajo de los subgráficos
		yanchor="top",
		#title_text=None,      # Sin título
		traceorder="normal",
		itemsizing="constant",
		font=dict(size=12),
		tracegroupgap=0,      # Ajusta el espaciado entre grupos
	),
	legend_tracegroupgap=20,
	margin=dict(l=40, r=60, t=120, b=100)
)

# Configuración de ejes personalizados
#fig.update_xaxes(title_text="Eje X (común)", row=2, col=1)

# Aplicar las mismas etiquetas de valores en el eje x a todos los subgráficos
for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_xaxes(
			tickangle=45, 
			tickfont=dict(size=12),
			tickvals=eje_x_etiquetas,           # Define los valores del eje x
			ticktext=eje_x_etiquetas,      # Define las etiquetas personalizadas para estos valores
			row=i, col=j
		)

#fig.update_xaxes(title_text="Eje X (común)", row=1, col=1)
#fig.update_yaxes(title_text="Variacion por 100", row=1, col=1)

for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_yaxes(
			#tickangle=45, 
			#tickfont=dict(size=12),
			#tickvals=eje_x_etiquetas,           # Define los valores del eje x
			#ticktext=eje_x_etiquetas,      # Define las etiquetas personalizadas para estos valores
			#row=i, col=j,

			title=dict(
				text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
				), type='linear', side='right', tickfont=dict(size=13),	row=i, col=j
		)

#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=1, col=1)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=1, col=2)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=2, col=1)
#fig.update_yaxes(title=dict(
#	text='Variacion por 100 respecto al inicio de la serie', font=dict(size=14), standoff=15
#	), type='linear', side='right', tickfont=dict(size=13),	row=2, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 2)", row=1, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 3)", row=2, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 4)", row=2, col=2)
#showlegend=True


fig.show()"""

In [ ]:
"""# Crear la cuadrícula de subgráficos: 2 filas x 2 columnas
fig = make_subplots(
	rows=2, cols=2, 
	subplot_titles=("Gráfico 1", "Gráfico 2", "Gráfico 3", "Gráfico 4")  # Títulos de cada subgráfico
)

# Datos de ejemplo para cada subgráfico
x_values = [1, 2, 3, 4, 5]
y_values1 = [10, 20, 30, 40, 50]
y_values2 = [50, 40, 30, 20, 10]
y_values3 = [15, 25, 35, 45, 55]
y_values4 = [5, 15, 25, 35, 45]

# Añadir el primer gráfico (Scatter) en la posición (1,1)
fig.add_trace(
	go.Scatter(x=x_values, y=y_values1, mode="lines+markers", name="Serie 1"), 
	row=1, col=1
)

# Añadir el segundo gráfico (Bar) en la posición (1,2)
fig.add_trace(
	go.Bar(x=x_values, y=y_values2, name="Serie 2"), 
	row=1, col=2
)

# Añadir el tercer gráfico (Scatter) en la posición (2,1)
fig.add_trace(
	go.Scatter(x=x_values, y=y_values3, mode="lines", name="Serie 3"), 
	row=2, col=1
)

# Añadir el cuarto gráfico (Bar) en la posición (2,2)
fig.add_trace(
	go.Bar(x=x_values, y=y_values4, name="Serie 4"), 
	row=2, col=2
)

# Configurar el layout general de la figura
fig.update_layout(
	title="Figura con Subgráficos",
	showlegend=True  # Mostrar leyenda compartida en la figura
)



fig.add_trace(go.Scatter(
		x=series_graf.index,
		y=series_graf[activos[0]],
		mode='lines',
		line=dict(color='blue', width=3, dash='dot'),
		opacity=0.8,
		name='Index'
	))


# Configuración de ejes personalizados
#fig.update_xaxes(title_text="Eje X (común)", row=2, col=1)
#fig.update_yaxes(title_text="Variacion por 100", row=1, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 2)", row=1, col=2)
#fig.update_yaxes(title_text="Eje Y (Serie 3)", row=2, col=1)
#fig.update_yaxes(title_text="Eje Y (Serie 4)", row=2, col=2)
#showlegend=True

# Mostrar el gráfico
fig.show()"""


"""
# Crear subgráficos con ejes compartidos y espaciado personalizado
fig = make_subplots(
	rows=2, cols=2, 
	shared_xaxes=True,  # Ejes x compartidos
	shared_yaxes=True,  # Ejes y compartidos
	vertical_spacing=0.1,  # Espacio vertical entre subgráficos
	horizontal_spacing=0.05,  # Espacio horizontal entre subgráficos
	subplot_titles=("Gráfico 1", "Gráfico 2", "Gráfico 3", "Gráfico 4")
)"""

In [ ]:
"""
# Datos de ejemplo
x_values = [1, 2, 3, 4, 5]  # Valores comunes en el eje x
series_graf_2.columns[0]

custom_labels = ["A", "B", "C", "D", "E"]  # Etiquetas personalizadas para el eje x
eje_x_etiquetas

y_values1 = [10, 20, 30, 40, 50]
y_values2 = [50, 40, 30, 20, 10]


# Aplicar las mismas etiquetas de valores en el eje x a todos los subgráficos
for i in range(1, 3): # Para filas 1 y 2
	for j in range(1, 3): # Para columnas 1 y 2
		fig.update_xaxes(
			tickvals=x_values,           # Define los valores del eje x
			ticktext=custom_labels,      # Define las etiquetas personalizadas para estos valores
			row=i, col=j
		)"""

# 4. Performance metrics

In [ ]:
ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
end_time = time.time()
execution_time = end_time - star_time

print(f'Correctly executed. Date: {ahora}.')
print(f'\nExecution time: {round(execution_time, 2)} seconds.')